In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfig:
  model_path: Path
  batch_size: int
  learning_rate: float
  epochs: int
  

In [5]:
import os
print(os.getcwd())
os.chdir("../")

c:\Users\HC107157\OneDrive - Qorvo\Documents\Qorvo\research


In [6]:
from src.SOC_Prediction.constants import *
from src.SOC_Prediction.utils.common import read_yaml,create_directories

ModuleNotFoundError: No module named 'box'

In [11]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model_config(self) -> PrepareModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        data_ingestion_config = PrepareModelConfig(
            model_path = config.model_path,
            batch_size = self.params.prepare_base_model.BATCH_SIZE,
            learning_rate = self.params.prepare_base_model.LR,
            epochs = self.params.prepare_base_model.EPOCHS 
        )

        return data_ingestion_config

In [9]:
import os
import pybamm
import numpy as np
import tensorflow as tf
from src.SOC_Prediction import logger

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config

    
    def get_model(self):
        model = tf.keras.Sequential(
            [tf.keras.layers.LSTM(10),
            tf.keras.layers.Dense(10,activation = 'relu'),
            tf.keras.layers.Dense(1,activation = 'linear')
            ]
        )
 
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )
        model.summary()
        self.save_model(path=self.config.model_path, model=self.model)
        return model


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

NameError: name 'read_yaml' is not defined